In [ ]:
!pip install dash

     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 1.0MB 18.6MB/s 
     |████████████████████████████████| 3.5MB 29.3MB/s 
     |████████████████████████████████| 194kB 55.3MB/s 
     |████████████████████████████████| 1.8MB 52.8MB/s 
     |████████████████████████████████| 358kB 49.2MB/s 
  Created wheel for dash: filename=dash-1.19.0-cp37-none-any.whl size=84011 sha256=505042afca2dc669e3b29c3f21eb0de17a4ab74c8ab1914928f665795871d0d9
  Stored in directory: /root/.cache/pip/wheels/f7/50/a7/a230ff7f503b10120bff18c2524a375bb85a61ce6b519c8a77
  Created wheel for flask-compress: filename=Flask_Compress-1.9.0-cp37-none-any.whl size=7259 sha256=6e6179025cf2f47bd20c09cec4f5bddb1edc8b084487a86506a7db2e22361ef9
  Stored in directory: /root/.cache/pip/wheels/0d/56/7d/e0c50541e5421b8291d958afb098c23818989e29d91a1edac5
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-cp37-none-any.whl size=1014870 sha256=124f994e05a2928fb4abf535b52dcb5d91dc

In [ ]:
!pip install  dash_bootstrap_components

     |████████████████████████████████| 194kB 12.5MB/s 


In [ ]:
!pip install jupyter_dash

In [21]:
from jupyter_dash import JupyterDash
import dash
import dash_html_components as htmlpy

from logging import debug
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash_html_components.Br import Br
from matplotlib.pyplot import scatter, title
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import scipy
from plotly.subplots import make_subplots
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from bsm_function import bsm
from sklearn.preprocessing import MinMaxScaler
import dash_bootstrap_components as dbc
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

def bsm(S, K, T, r, sigma):
    # S: spot price
    # K: strike price
    # T: time to maturity
    # r: interest rate
    # sigma: volatility of underlying asset
    global call
    d1 = ((np.log(S / K) + (r + 0.5 * sigma ** 2) * T)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * scipy.stats.norm(0, 1).cdf(d1) - K * np.exp(-r * T) * scipy.stats.norm(
        0, 1
    ).cdf(d2)
    # print( "D1 è:", d1,"\n"
    #      "D2 é:",d2, "\n", "\n")
    return call


# https://plotly.com/python/ml-regression/#ml-regression-in-dash

# parte per il bsm

df_filtered_bsm = pd.read_csv("/content/drive/MyDrive/File_tesi/df_filtered_bsm.csv")
#df_filtered_bsm = pd.read_csv("df_filtered_bsm.csv")
df_filtered_bsm = df_filtered_bsm.drop("Unnamed: 0", axis=1)
df_filtered_bsm.head()


df_filtered_bsm_s = df_filtered_bsm.sample(frac=0.2)
df_filtered_bsm_s = df_filtered_bsm_s.reset_index(drop=True)

#%%
BSM1 = list()
for i in range(len(df_filtered_bsm_s)):

    BSM1.append(
        bsm(
            df_filtered_bsm_s.loc[i, "Price"],
            df_filtered_bsm_s.loc[i, "Strike"],
            df_filtered_bsm_s.loc[i, "DTE in years"],
            df_filtered_bsm_s.loc[i, "Risk free"],
            df_filtered_bsm_s.loc[i, "IV_brenner"] / 100,
        )
    )
BSM1 = pd.DataFrame(BSM1, columns=["Results"])
df_filtered_bsm_r = pd.concat([df_filtered_bsm_s, BSM1], axis=1)

# Parte per il Random Forest
df_filtered_rf = df_filtered_bsm.drop(
    [
        "Symbol",
        "Type",
        "Time",
        "Exp Date",
        "Bid",
        "Last",
        "Volume",
        "Open Int",
        "Vol/OI",
        "IV",
        "DTE",
    ],
    axis=1,
)

X = df_filtered_rf.drop("Ask", axis=1)
y = df_filtered_rf["Ask"]


models = {
    "BSM": "linear_model.LinearRegression",
    "RFF": "tree.DecisionTreeRegressor",
    "ANN": "Neural Network",
    "KNN": "K-nearest-neighourd",
    
}

app = JupyterDash(__name__)

app.layout = html.Div(
    [
        html.H1("Select Model:"),
        dcc.Dropdown(
            id="model-name",
            options=[{"label": x, "value": x} for x in models],
            value="BSM",
            clearable=False,
        ),
        html.Br(),
        html.Div(id="header"),
        # html.Br(),
        dcc.Graph(id="graph"),
    ]
)


@app.callback(
    Output("graph", "figure"),
    Output("header", "children"),
    [Input("model-name", "value")],
)
def train_and_display(name):
    if name == "BSM":

        fig = make_subplots(rows=1, cols=2)

        fig.add_trace(
            go.Scatter(
                name="Real vs Predicted price",
                x=df_filtered_bsm_r["Ask"],
                y=df_filtered_bsm_r["Results"],
                mode="markers",
            ),
            row=1,
            col=1,
        )
        fig.update_xaxes(range=[0, 450], row=1, col=1, title="Real Ask price in $"),
        fig.update_yaxes(
            row=1, col=1, title="{} Predicted ask price in $".format(name)
        ),

        fig.add_trace(
            go.Scatter(
                name="45° Degree line",
                x=df_filtered_bsm_r["Ask"],
                y=df_filtered_bsm_r["Ask"],
            ),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Histogram(
                name="Errors",
                x=(df_filtered_bsm_r["Results"] - df_filtered_bsm_r["Ask"]),
                marker=dict(color="#257EC4"),
            ),
            row=1,
            col=2,
        )
        fig.update_xaxes(
            range=[-100, 200],
            row=1,
            col=2,
            title="{} errors histogram in $".format(name),
        ),

        fig.update_layout(height=500)
        global stringa
        stringa = "Graphs generated by Black & Scholes equation, approximating IV with Brenner and Subrahmanyan 1988 formula"
        return fig, stringa

    if name == "RFF":
        model = RandomForestRegressor(
            max_depth=15,
            max_features=3,
            min_samples_leaf=2,
            min_samples_split=4,
            n_estimators=30,
        )

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.20, random_state=42
        )
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        fig = make_subplots(rows=1, cols=2)

        fig.add_trace(
            go.Scatter(
                name="Real vs Predicted price", x=y_test, y=predictions, mode="markers"
            ),
            row=1,
            col=1,
        )
        fig.update_xaxes(range=[0, 450], row=1, col=1, title="Real Ask price in $"),
        fig.update_yaxes(
            row=1, col=1, title="{} Predicted ask price in $".format(name)
        ),

        fig.add_trace(
            go.Scatter(name="45° Degree line", x=y_test, y=y_test), row=1, col=1
        )

        fig.add_trace(
            go.Histogram(
                name="Errors", x=(predictions - y_test), marker=dict(color="#257EC4")
            ),
            row=1,
            col=2,
        )
        fig.update_xaxes(
            range=[-40, 40], row=1, col=2, title="{} errors histogram in $".format(name)
        ),

        fig.update_layout(height=500)
        stringa = "Graphs generated by Random Forest algorithm with 30 estimator and max depth of 15"
        return fig, stringa

    if name == "KNN":
        X_train_knn, X_test_knn, y_train, y_test = train_test_split(
            X, y, test_size=0.20, random_state=42
        )

        scaler = MinMaxScaler()
        X_train_knn = scaler.fit_transform(X_train_knn)
        X_test_knn = scaler.transform(X_test_knn)

        model = KNeighborsRegressor(
            metric="euclidean", n_neighbors=2, p=1, weights="distance"
        )

        model.fit(X_train_knn, y_train)
        predictions = model.predict(X_test_knn)

        fig = make_subplots(rows=1, cols=2)

        fig.add_trace(
            go.Scatter(
                name="Real vs Predicted price", x=y_test, y=predictions, mode="markers"
            ),
            row=1,
            col=1,
        )
        fig.update_xaxes(range=[0, 450], row=1, col=1, title="Real Ask price in $"),
        fig.update_yaxes(
            row=1, col=1, title="{} Predicted ask price in $".format(name)
        ),

        fig.add_trace(
            go.Scatter(name="45° Degree line", x=y_test, y=y_test), row=1, col=1
        )

        fig.add_trace(
            go.Histogram(
                name="Errors", x=(predictions - y_test), marker=dict(color="#257EC4")
            ),
            row=1,
            col=2,
        )
        fig.update_xaxes(
            range=[-40, 40], row=1, col=2, title="{} errors histogram in $".format(name)
        ),

        fig.update_layout(height=500)

        stringa = (
            "Graphs generated by KNN algorithm with a number of neighbors equal to 2"
        )

        return fig, stringa
    if name == "ANN":
        # per caricare il modello.
        model = load_model("/content/drive/MyDrive/File_tesi/model.h5")
        #model = load_model("model.h5")
        X_train_ann, X_test_ann, y_train, y_test = train_test_split(
            X, y, test_size=0.20, random_state=42
        )

        scaler = MinMaxScaler()
        X_train_ann = scaler.fit_transform(X_train_ann)
        X_test_ann = scaler.transform(X_test_ann)

        predictions = model.predict(X_test_ann)
        predictions = predictions.reshape((737,))
        fig = make_subplots(rows=1, cols=2)

        fig.add_trace(
            go.Scatter(
                name="Real vs Predicted price", x=y_test, y=predictions, mode="markers"
            ),
            row=1,
            col=1,
        )
        fig.update_xaxes(range=[0, 450], row=1, col=1, title="Real Ask price in $"),
        fig.update_yaxes(
            row=1, col=1, title="{} Predicted ask price in $".format(name)
        ),

        fig.add_trace(
            go.Scatter(name="45° Degree line", x=y_test, y=y_test), row=1, col=1
        )

        fig.add_trace(
            go.Histogram(
                name="Errors", x=(predictions - y_test), marker=dict(color="#257EC4")
            ),
            row=1,
            col=2,
        )
        fig.update_xaxes(
            range=[-40, 40], row=1, col=2, title="{} errors histogram in $".format(name)
        ),

        fig.update_layout(height=500)

        stringa = "Graphs generated by Neural Networks algorithm with 3 hidden layers with 12,6, 3 nodes respectively"

        return fig, stringa

app.run_server(mode="external")


Dash app running on:


<IPython.core.display.Javascript object>